In [2]:
import tkinter

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import math

# --- 1. Distance Calculation Function (Euclidean Distance) ---
def calculate_distance(p1, p2):
    """Calculates the 2D Euclidean distance between two points (x, y)."""
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# --- 2. Initialize MediaPipe and OpenCV ---
mp_hands = mp.solutions.hands
# Set up the Hand detection object
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils # Utility for drawing landmarks

cap = cv2.VideoCapture(0) # Open the default webcam (index 0)

# --- 3. Main Loop for Frame Processing ---
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Ignoring empty camera frame.")
        continue

    # Flip the frame horizontally for a natural, mirror-like view
    frame = cv2.flip(frame, 1)
    
    # Get frame dimensions
    h, w, c = frame.shape

    # Convert the BGR image to RGB for MediaPipe
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame to find hand landmarks
    results = hands.process(img_rgb)
    
    # --- 4. Process Landmarks and Calculate Distance ---
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            
            # Draw the full set of 21 landmarks and connections
            mp_draw.draw_landmarks(
                frame, 
                hand_landmarks, 
                mp_hands.HAND_CONNECTIONS,
                mp_draw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2), # Landmark color
                mp_draw.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2) # Connection color
            )

            # Get coordinates for Thumb Tip (Landmark ID 4) and Little Finger Tip (Landmark ID 20)
            
            # THUMB TIP (ID 4)
            lm_4 = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            x4, y4 = int(lm_4.x * w), int(lm_4.y * h)
            
            # LITTLE FINGER TIP (ID 20)
            lm_20 = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
            x20, y20 = int(lm_20.x * w), int(lm_20.y * h)
            
            # Highlight the specific points
            cv2.circle(frame, (x4, y4), 10, (255, 0, 0), cv2.FILLED)  # Blue circle for thumb
            cv2.circle(frame, (x20, y20), 10, (0, 0, 255), cv2.FILLED) # Red circle for pinky
            
            # Draw a line connecting the points
            cv2.line(frame, (x4, y4), (x20, y20), (0, 255, 255), 3) # Yellow line
            
            # Calculate the distance
            distance = calculate_distance((x4, y4), (x20, y20))
            
            # Display the distance on the frame
            cv2.putText(
                frame, 
                f'Distance: {int(distance)} px', 
                (20, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (255, 255, 255), 
                2, 
                cv2.LINE_AA
            )
            
    # --- 5. Display the output frame ---
    cv2.imshow('Hand Gesture Volume Control', frame)
    
    # Exit loop on 'q' press
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# --- 6. Cleanup ---
cap.release()
cv2.destroyAllWindows()

In [7]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
from google.protobuf.json_format import MessageToDict 

# --- MediaPipe Setup (No change) ---
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    max_num_hands=2
)

# Landmark indices for Tips (No change)
THUMB_TIP_IDX = 4
FINGER_TIPS_INDICES = {
    "Index Finger": 8,
    "Middle Finger": 12,
    "Ring Finger": 16,
    "Pinky Finger": 20,
}

# --- Distance Calculation Function (No change) ---
def calculate_distance(df, lm_a_idx, lm_b_idx):
    """Calculates the 3D normalized distance between two landmarks."""
    lm_a = df.loc[lm_a_idx]
    lm_b = df.loc[lm_b_idx]
    norm_dist = np.sqrt(
        (lm_a["x_norm"] - lm_b["x_norm"])**2 +
        (lm_a["y_norm"] - lm_b["y_norm"])**2 +
        (lm_a["z_norm"] - lm_b["z_norm"])**2
    )
    return norm_dist

# --- Tkinter Application Class (Updated update_video) ---
class HandTrackingApp:
    def __init__(self, master):
        self.master = master
        master.title("✋ Hand Gesture Analyzer (Live Feed Fixed)")
        master.geometry("1000x700")
        master.configure(bg="#f4f7f6")

        # --- Frames for layout ---
        self.main_frame = ttk.Frame(master, padding="15 15 15 15")
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        self.video_frame = ttk.LabelFrame(self.main_frame, text="Live Hand Tracking", padding="10 10 10 10")
        self.video_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.data_frame = ttk.Frame(self.main_frame, padding="10 10 10 10", width=300)
        self.data_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)
        
        # Handedness Header
        self.handedness_label = ttk.Label(self.data_frame, text="Detecting Hand...", font=('Arial', 16, 'bold'), foreground='navy')
        self.handedness_label.pack(anchor='w', pady=(0, 15))

        # --- Video Canvas (Display) ---
        self.video_label = tk.Label(self.video_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        # --- Data Labels ---
        self.distance_labels = {}
        for finger_name in FINGER_TIPS_INDICES.keys():
            label_text = f"↔️ {finger_name} Distance:"
            label = ttk.Label(self.data_frame, text=label_text, font=('Arial', 12, 'bold'))
            label.pack(anchor='w', pady=(10, 2))
            
            value_label = ttk.Label(self.data_frame, text="--.-- (Normalized)", font=('Courier', 14))
            value_label.pack(anchor='w', pady=(0, 10))
            self.distance_labels[finger_name] = value_label

        # --- Status Label ---
        self.status_label = ttk.Label(self.data_frame, text="Status: Initializing...", font=('Arial', 10), foreground='gray')
        self.status_label.pack(anchor='s', pady=(20, 0))

        # --- Webcam Setup ---
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            self.status_label.config(text="Status: CAMERA ERROR! Check connection or permissions.", foreground='red')
            return
        
        self.hands_processor = hands 
        self.update_video()

    def update_video(self):
        ret, frame = self.cap.read()
        
        if ret:
            frame = cv2.flip(frame, 1)
            img_h, img_w = frame.shape[:2]
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            results = self.hands_processor.process(frame_rgb)
            
            annotated_frame = frame.copy()
            status_text = "No Hand Detected"
            status_color = 'orange'
            
            self.handedness_label.config(text="No Hand Detected", foreground='navy')

            if results.multi_hand_landmarks and len(results.multi_hand_landmarks) > 0:
                hand_landmarks = results.multi_hand_landmarks[0]
                
                handedness_proto = results.multi_handedness[0]
                handedness_dict = MessageToDict(handedness_proto)
                handedness = handedness_dict['classification'][0]['label'] 
                
                self.handedness_label.config(text=f"Hand Detected: {handedness}", 
                                             foreground='darkgreen' if handedness == 'Right' else 'darkred')
                
                mp_drawing.draw_landmarks(
                    annotated_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 100), thickness=2, circle_radius=4), 
                    mp_drawing.DrawingSpec(color=(200, 200, 255), thickness=2, circle_radius=2)
                )

                rows = []
                for idx, lm in enumerate(hand_landmarks.landmark):
                    rows.append({
                        "index": idx,
                        "x_norm": lm.x, "y_norm": lm.y, "z_norm": lm.z,
                        "x_px": int(lm.x * img_w), "y_px": int(lm.y * img_h)
                    })
                df = pd.DataFrame(rows).set_index("index")

                status_text = f"Tracking {handedness} Hand"
                status_color = 'green'
                
                for finger_name, tip_idx in FINGER_TIPS_INDICES.items():
                    if tip_idx in df.index and THUMB_TIP_IDX in df.index:
                        norm_dist = calculate_distance(df, THUMB_TIP_IDX, tip_idx)
                        self.distance_labels[finger_name].config(text=f"{norm_dist:.4f} (Normalized)", foreground='green')
                        
                        thumb_tip = df.loc[THUMB_TIP_IDX]
                        finger_tip = df.loc[tip_idx]
                        cv2.line(annotated_frame, 
                                 (thumb_tip["x_px"], thumb_tip["y_px"]),
                                 (finger_tip["x_px"], finger_tip["y_px"]),
                                 (0, 255, 0), 2)
                    else:
                        self.distance_labels[finger_name].config(text="--.-- (Missing Landmark)", foreground='red')
            
            else:
                for label in self.distance_labels.values():
                    label.config(text="--.-- (Normalized)", foreground='gray')


            self.status_label.config(text=f"Status: {status_text}", foreground=status_color)

            # Convert to PIL Image for Tkinter
            img = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            
            width, height = self.video_label.winfo_width(), self.video_label.winfo_height()
            if width > 10 and height > 10: 
                img = img.resize((width, height), Image.Resampling.LANCZOS)
            
            # --- THE FIX APPLIED HERE ---
            self.photo = ImageTk.PhotoImage(image=img)
            self.video_label.config(image=self.photo)
            self.video_label.image = self.photo # IMPORTANT: Keep the reference!

        # Loop the update function
        self.master.after(10, self.update_video) 

    def on_closing(self):
        print("Closing application and releasing webcam...")
        self.cap.release()
        self.hands_processor.close()
        self.master.destroy()

# --- Main Execution (No change) ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        style = ttk.Style(root)
        style.theme_use('clam') 
        
        app = HandTrackingApp(root)
        root.protocol("WM_DELETE_WINDOW", app.on_closing)
        root.mainloop()
    except Exception as e:
        print(f"An error occurred: {e}")
        hands.close()

An error occurred: image "pyimage67" doesn't exist


In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
# This import is required to correctly parse MediaPipe's handedness data
from google.protobuf.json_format import MessageToDict 

# --- MediaPipe Setup ---
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    max_num_hands=2
)

# Landmark indices for Tips
THUMB_TIP_IDX = 4
FINGER_TIPS_INDICES = {
    "Index Finger": 8,
    "Middle Finger": 12,
    "Ring Finger": 16,
    "Pinky Finger": 20,
}

# --- Distance Calculation Function ---
def calculate_distance(df, lm_a_idx, lm_b_idx):
    """Calculates the 3D normalized distance between two landmarks."""
    lm_a = df.loc[lm_a_idx]
    lm_b = df.loc[lm_b_idx]
    norm_dist = np.sqrt(
        (lm_a["x_norm"] - lm_b["x_norm"])**2 +
        (lm_a["y_norm"] - lm_b["y_norm"])**2 +
        (lm_a["z_norm"] - lm_b["z_norm"])**2
    )
    return norm_dist

# --- Tkinter Application Class ---
class HandTrackingApp:
    def __init__(self, master):
        self.master = master
        master.title("✋ Hand Gesture Analyzer (Live Feed Fixed)")
        master.geometry("1000x700")
        master.configure(bg="#f4f7f6")
        
        # Initialize self.photo here for robustness against GC
        self.photo = None 

        # --- Frames for layout ---
        self.main_frame = ttk.Frame(master, padding="15 15 15 15")
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        self.video_frame = ttk.LabelFrame(self.main_frame, text="Live Hand Tracking", padding="10 10 10 10")
        self.video_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.data_frame = ttk.Frame(self.main_frame, padding="10 10 10 10", width=300)
        self.data_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)
        
        # Handedness Header
        self.handedness_label = ttk.Label(self.data_frame, text="Detecting Hand...", font=('Arial', 16, 'bold'), foreground='navy')
        self.handedness_label.pack(anchor='w', pady=(0, 15))

        # --- Video Canvas (Display) ---
        self.video_label = tk.Label(self.video_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        # --- Data Labels ---
        self.distance_labels = {}
        for finger_name in FINGER_TIPS_INDICES.keys():
            label_text = f"↔️ {finger_name} Distance:"
            label = ttk.Label(self.data_frame, text=label_text, font=('Arial', 12, 'bold'))
            label.pack(anchor='w', pady=(10, 2))
            
            value_label = ttk.Label(self.data_frame, text="--.-- (Normalized)", font=('Courier', 14))
            value_label.pack(anchor='w', pady=(0, 10))
            self.distance_labels[finger_name] = value_label

        # --- Status Label ---
        self.status_label = ttk.Label(self.data_frame, text="Status: Initializing...", font=('Arial', 10), foreground='gray')
        self.status_label.pack(anchor='s', pady=(20, 0))

        # --- Webcam Setup ---
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            self.status_label.config(text="Status: CAMERA ERROR! Check connection or permissions.", foreground='red')
            # Use after to prevent immediate exit if camera fails
            self.master.after(100, lambda: self.status_label.config(text="Status: CAMERA ERROR!", foreground='red')) 
            return
        
        self.hands_processor = hands 
        self.update_video()

    def update_video(self):
        ret, frame = self.cap.read()
        
        if ret:
            frame = cv2.flip(frame, 1)
            img_h, img_w = frame.shape[:2]
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            results = self.hands_processor.process(frame_rgb)
            
            annotated_frame = frame.copy()
            status_text = "No Hand Detected"
            status_color = 'orange'
            
            self.handedness_label.config(text="No Hand Detected", foreground='navy')

            if results.multi_hand_landmarks and len(results.multi_hand_landmarks) > 0:
                hand_landmarks = results.multi_hand_landmarks[0]
                
                handedness_proto = results.multi_handedness[0]
                handedness_dict = MessageToDict(handedness_proto)
                handedness = handedness_dict['classification'][0]['label'] 
                
                self.handedness_label.config(text=f"Hand Detected: {handedness}", 
                                             foreground='darkgreen' if handedness == 'Right' else 'darkred')
                
                mp_drawing.draw_landmarks(
                    annotated_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 100), thickness=2, circle_radius=4), 
                    mp_drawing.DrawingSpec(color=(200, 200, 255), thickness=2, circle_radius=2)
                )

                rows = []
                for idx, lm in enumerate(hand_landmarks.landmark):
                    rows.append({
                        "index": idx,
                        "x_norm": lm.x, "y_norm": lm.y, "z_norm": lm.z,
                        "x_px": int(lm.x * img_w), "y_px": int(lm.y * img_h)
                    })
                df = pd.DataFrame(rows).set_index("index")

                status_text = f"Tracking {handedness} Hand"
                status_color = 'green'
                
                for finger_name, tip_idx in FINGER_TIPS_INDICES.items():
                    if tip_idx in df.index and THUMB_TIP_IDX in df.index:
                        norm_dist = calculate_distance(df, THUMB_TIP_IDX, tip_idx)
                        self.distance_labels[finger_name].config(text=f"{norm_dist:.4f} (Normalized)", foreground='green')
                        
                        thumb_tip = df.loc[THUMB_TIP_IDX]
                        finger_tip = df.loc[tip_idx]
                        cv2.line(annotated_frame, 
                                 (thumb_tip["x_px"], thumb_tip["y_px"]),
                                 (finger_tip["x_px"], finger_tip["y_px"]),
                                 (0, 255, 0), 2)
                    else:
                        self.distance_labels[finger_name].config(text="--.-- (Missing Landmark)", foreground='gray')
            
            else:
                for label in self.distance_labels.values():
                    label.config(text="--.-- (Normalized)", foreground='gray')


            self.status_label.config(text=f"Status: {status_text}", foreground=status_color)

            # Convert to PIL Image for Tkinter
            img = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            
            width, height = self.video_label.winfo_width(), self.video_label.winfo_height()
            if width > 10 and height > 10: 
                img = img.resize((width, height), Image.Resampling.LANCZOS)
            
            # THE FINAL FIX: Storing the reference on self.photo ensures persistence.
            self.photo = ImageTk.PhotoImage(image=img)
            self.video_label.config(image=self.photo)
            
        # Loop the update function
        self.master.after(10, self.update_video) 

    def on_closing(self):
        print("Closing application and releasing webcam...")
        self.cap.release()
        self.hands_processor.close()
        self.master.destroy()

# --- Main Execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        style = ttk.Style(root)
        style.theme_use('clam') 
        
        app = HandTrackingApp(root)
        root.protocol("WM_DELETE_WINDOW", app.on_closing)
        root.mainloop()
    except Exception as e:
        print(f"An external error occurred: {e}")
        hands.close()

C:\Users\acer\AppData\Roaming\Python\Python310\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\acer\.conda\envs\cv_project\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\acer\.conda\envs\cv_project\lib\tkinter\__init__.py", line 839, in callit
    func(*args)
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_20104\2394491618.py", line 150, in update_video
    cv2.line(annotated_frame,
cv2.error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type



Closing application and releasing webcam...
